In [1]:
import pandas as pd
import numpy as np

In [2]:
on_faults = pd.read_pickle('../data/on_faults.pkl')

In [3]:
on_faults.dtypes

RecordID                              int64
ESS_Id                                int64
EventTimeStamp               datetime64[ns]
eventDescription                     object
ecuSoftwareVersion                   object
ecuSerialNumber                      object
ecuModel                             object
ecuMake                              object
ecuSource                             int64
spn                                   int64
fmi                                   int64
active                                 bool
activeTransitionCount                 int64
EquipmentID                          object
MCTNumber                             int64
Latitude                            float64
Longitude                           float64
LocationTimeStamp            datetime64[ns]
EventDate                    datetime64[ns]
EventTime                            object
LocationDate                 datetime64[ns]
LocationTime                         object
AcceleratorPedal                

So we need all readouts of codes other than the breakdown down within a time period of 1 week before the breakdown code occured. 

WE will have to get some sort of aggregate of events occuring up to the derate and then make new binary features of weather or not they occured. Then our result is whether or not the truck derates.  

Each column of the df will be time periods:

7-6 days prior total # of non derate codes (ie not 5246)

6-5 '''

etc.

In [4]:
window_size = pd.Timedelta('1D')

In [5]:
df = on_faults.set_index('EventTimeStamp').sort_index(ascending = False)

Make dummy columns for category of spn, get_dummies function

In [7]:
df = (pd.get_dummies(data = df, columns =['spn'])
        .filter(regex='^(spn|EquipmentID)'))

In [8]:
derate_times = df[df['spn_5246'] == 1][['EquipmentID', 'spn_5246']] 

In [9]:
agg_dict = {col: 'sum' for col in df.columns if (col != 'EquipmentID')}

In [10]:
agg_dict['EquipmentID'] = lambda x : x[-1]

In [11]:
rolling = (df
            .groupby(by = 'EquipmentID')
            .rolling(window = '1D')
            .sum()         

)

In [12]:
rolling

spn_0  spn_16  spn_27  spn_33  spn_37  \
EquipmentID EventTimeStamp                                               
301         2020-02-20 12:11:55    0.0     0.0     0.0     0.0     0.0   
            2020-02-20 12:09:01    0.0     0.0     0.0     0.0     0.0   
            2020-02-10 13:39:13    0.0     0.0     0.0     0.0     0.0   
            2020-02-10 13:35:03    0.0     0.0     0.0     0.0     0.0   
            2020-02-10 10:51:35    0.0     0.0     0.0     0.0     0.0   
...                                ...     ...     ...     ...     ...   
R1762       2015-02-24 15:31:56    0.0     0.0     0.0     0.0     0.0   
            2015-02-24 15:31:17    0.0     0.0     0.0     0.0     0.0   
            2015-02-24 13:45:06    0.0     0.0     0.0     0.0     0.0   
R1764       2015-02-25 06:38:40    0.0     0.0     0.0     0.0     0.0   
            2015-02-25 06:08:43    0.0     0.0     0.0     0.0     0.0   

                                 spn_38  spn_51  spn_70  spn_74  spn_75  ...  \
EquipmentID EventTimeStamp                                               ...   
301         2020-02-20 12:11:55     0.0     0.0     0.0     0.0     0.0  ...   
            2020-02-20 12:09:01     0.0     0.0     0.0     0.0     0.0  ...   
            2020-02-10 13:39:13     0.0     0.0     0.0     0.0     0.0  ...   
            2020-02-10 13:35:03     0.0     0.0     0.0     0.0     0.0  ...   
            2020-02-10 10:51:35     0.0     0.0     0.0     0.0     0.0  ...   
...                                 ...     ...     ...     ...     ...  ...   
R1762       2015-02-24 15:31:56     0.0     0.0     0.0     0.0     0.0  ...   
            2015-02-24 15:31:17     0.0     0.0     0.0     0.0     0.0  ...   
            2015-02-24 13:45:06     0.0     0.0     0.0     0.0     0.0  ...   
R1764       2015-02-25 06:38:40     0.0     0.0     0.0     0.0     0.0  ...   
            2015-02-25 06:08:43     0.0     0.0     0.0     0.0     0.0  ...   

                                 spn_520413  spn_520953  spn_521032  \
EquipmentID EventTimeStamp                                            
301         2020-02-20 12:11:55         0.0         0.0         0.0   
            2020-02-20 12:09:01         0.0         0.0         0.0   
            2020-02-10 13:39:13         0.0         0.0         0.0   
            2020-02-10 13:35:03         0.0         0.0         0.0   
            2020-02-10 10:51:35         0.0         0.0         0.0   
...                                     ...         ...         ...   
R1762       2015-02-24 15:31:56         0.0         0.0         0.0   
            2015-02-24 15:31:17         0.0         0.0         0.0   
            2015-02-24 13:45:06         0.0         0.0         0.0   
R1764       2015-02-25 06:38:40         0.0         0.0         0.0   
            2015-02-25 06:08:43         0.0         0.0         0.0   

                                 spn_523530  spn_523531  spn_523543  \
EquipmentID EventTimeStamp                                            
301         2020-02-20 12:11:55         0.0         0.0         0.0   
            2020-02-20 12:09:01         0.0         0.0         0.0   
            2020-02-10 13:39:13         0.0         0.0         0.0   
            2020-02-10 13:35:03         0.0         0.0         0.0   
            2020-02-10 10:51:35         0.0         0.0         0.0   
...                                     ...         ...         ...   
R1762       2015-02-24 15:31:56         0.0         0.0         0.0   
            2015-02-24 15:31:17         0.0         0.0         0.0   
            2015-02-24 13:45:06         0.0         0.0         0.0   
R1764       2015-02-25 06:38:40         0.0         0.0         0.0   
            2015-02-25 06:08:43         0.0         0.0         0.0   

                                 spn_524033  spn_524037  spn_524071  \
EquipmentID EventTimeStamp                                            
301         2020-02-20 12:11:55        

2 column ID for nth derate

class imbalance 